# QC: Results match Capacity Assumptions from TWG meeting and Board

Board of Directors Meeting MAY 25, 2018

### 509,000 capacity

https://www.sandag.org/uploads/meetingid/meetingid_4785_23865.pdf (page 852)

In [1]:
import os
import sys

In [2]:
# append path to find utils module in urbansim
cwd = os.getcwd() 
parentdir =  os.path.abspath(os.path.join(cwd, os.pardir))
parentdir2 =  os.path.abspath(os.path.join(parentdir, os.pardir))
sys.path.append(parentdir2) # to get path to utils module

In [3]:
import pandas as pd
import numpy as np
import utils

In [4]:
from sqlalchemy import create_engine
from database import get_connection_string

In [5]:
%matplotlib inline

In [6]:
# connect to database
db_connection_string = get_connection_string('..\..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

In [7]:
# get versions
versions = utils.yaml_to_dict('../../data/scenario_config.yaml', 'scenario')

### Get board report capacity assumptions

In [8]:
d = {'jur_id': range(1,20),\
    'twg2016to2035': [6600, 27700,200,100,3200,2500,11800,3600,9300,1100,5800,6900,1500,202600,11500,3300,400,7100,67800],\
     'twg2036to2050': [2100, 14200,200,200,7400,1100,8800,300,1900,1500,6400,5200,2000,65600,4300,3400,200,3400,7800]}
dfboard = pd.DataFrame(data=d)
#dfboard['jur_id'] = range(1,20)
# dfboard

In [9]:
dfboard['twgTotal'] = dfboard.iloc[:, 1:3].sum(1)

In [10]:
# dfboard

# Get geographies

In [11]:
lookup_sql = '''
SELECT parcel_id,jur_id
FROM [isam].[xpef04].[parcel2015_mgra_jur_cpa] 
WHERE  i=1'''
lookup_df = pd.read_sql(lookup_sql, mssql_engine)
jur_name_sql = '''SELECT [jurisdiction_id] as jur_id,[name] as jur_name FROM [urbansim].[ref].[jurisdiction]'''
jur_name = pd.read_sql(jur_name_sql,mssql_engine)
lookup_df = pd.merge(lookup_df,jur_name,on='jur_id',how='left')

# Get units added

In [12]:
# get max run id from urbansim
run_id_sql = '''
SELECT max(run_id)
  FROM [urbansim].[urbansim].[urbansim_lite_output]
'''
run_id_df = pd.read_sql(run_id_sql, mssql_engine)
run_id = int(run_id_df.values)
print("\n   Max run id : {:,}".format(run_id))


   Max run id : 411


In [13]:
# run_id = input()

In [14]:
output_sql = '''
    SELECT parcel_id,year_simulation,unit_change
    FROM urbansim.urbansim.urbansim_lite_output urbansim_lite_output 
    WHERE  run_id =  %s'''
output_sql = output_sql % run_id
output = pd.read_sql(output_sql,mssql_engine)

In [15]:
print("units added sum: {:,}".format(output.unit_change.sum()))

units added sum: 418,845


## Classify units into two categories based on year added: "2016 to 2035" & "2036 to 2050"

In [16]:
output['yr_category'] = 'urb2016to2035'

In [17]:
output.loc[output['year_simulation']>2035,'yr_category'] = 'urb2036to2050' 

##  Add city to output dataframe

In [18]:
output = pd.merge(output,lookup_df,on='parcel_id',how='left')

In [19]:
# output.head()

## Group by jurisdiction and year category and sum unit change

In [20]:
df = pd.DataFrame({'unit_change': output.groupby(["jur_name","jur_id","yr_category"])
                                          .unit_change.sum()}).reset_index()

In [21]:
df.unit_change.sum()
#419,558

418845

In [22]:
# df.head()

## Pivot data so year category is a column and jurisdiction is a row

In [23]:
table = pd.pivot_table(df, values='unit_change', index=[ 'jur_id','jur_name'],columns=['yr_category'])

In [24]:
table.reset_index(inplace=True)

In [25]:
table.columns.name = None

In [26]:
# table

In [27]:
table['urbTotal'] = table.iloc[:, 2:4].sum(1)

In [28]:
table.urbTotal.sum()
#419558

418845

In [29]:
table_w_boardreport = pd.merge(dfboard,table,on='jur_id',how='left')

In [30]:
table_w_boardreport.loc['Total']= table_w_boardreport.sum()

In [31]:
table_w_boardreport.loc[table_w_boardreport.jur_id==190,'jur_name'] = 'Region'

In [32]:
table_w_boardreport['jur_id'] = table_w_boardreport['jur_id'].astype('int')

In [33]:
table_w_boardreport.loc[table_w_boardreport.jur_id==190,'jur_id'] = 'Total'

In [34]:
table_w_boardreport.set_index(['jur_id'],inplace=True)

In [35]:
# table_w_boardreport

In [36]:
t2016to2035 = table_w_boardreport[['jur_name','twg2016to2035','urb2016to2035']].copy()

In [37]:
t2016to2035['remaining2016to2035'] = t2016to2035['twg2016to2035'] - t2016to2035['urb2016to2035']

In [38]:
# t2016to2035

In [39]:
num_remaining2016to2035 = len(t2016to2035.loc[t2016to2035.remaining2016to2035<0])

In [40]:
t2036to2050 = table_w_boardreport[['jur_name','twg2036to2050','urb2036to2050']].copy()

In [41]:
t2036to2050['remaining2036to2050'] = t2036to2050['twg2036to2050'] - t2036to2050['urb2036to2050']

In [42]:
t2016to2050 = table_w_boardreport[['jur_name','twg2016to2035','urb2016to2035','twg2036to2050','urb2036to2050']].copy()
t2016to2050['remaining2016to2035'] = t2016to2050['twg2016to2035'] - t2016to2050['urb2016to2035']
t2016to2050['remaining2036to2050'] = t2016to2050['twg2036to2050'] - t2016to2050['urb2036to2050']

In [43]:
# t2036to2050

In [44]:
num_remaining2036to2050 = len(t2036to2050.loc[t2036to2050.remaining2036to2050<0])

In [45]:
tTotal = table_w_boardreport[['jur_name','twgTotal','urbTotal']].copy()

In [46]:
tTotal['remaining'] = tTotal['twgTotal'] - tTotal['urbTotal']

In [47]:
# tTotal

In [48]:
num_remaining = len(tTotal.loc[tTotal.remaining<0])

In [49]:
print('From Board Report with 509,000 capacity')
table_w_boardreport[['jur_name','twg2016to2035','twg2036to2050','twgTotal','urb2016to2035','urb2036to2050','urbTotal']]

From Board Report with 509,000 capacity


,jur_name,twg2016to2035,twg2036to2050,twgTotal,urb2016to2035,urb2036to2050,urbTotal
jur_id,,,,,,,
1,Carlsbad,6600,2100,8700,5173,2468,7641
2,Chula Vista,27700,14200,41900,9214,21786,31000
3,Coronado,200,200,400,106,291,397
4,Del Mar,100,200,300,44,84,128
5,El Cajon,3200,7400,10600,2917,1937,4854
6,Encinitas,2500,1100,3600,1442,1652,3094
7,Escondido,11800,8800,20600,9183,5500,14683
8,Imperial Beach,3600,300,3900,2894,674,3568
9,La Mesa,9300,1900,11200,7653,2705,10358


#### Note: over the entire forecast period all cities are below the capacity assumptions in the board report, except Unincorporated

#### Remaining capacity in the first increment (2016 to 2035) is used in the second increment (2036 to 2050)

#### Thus the total capacity used in the later period (2036 to 2050) is greater than the board report

In [50]:
if (num_remaining + num_remaining2036to2050 + num_remaining2016to2035) == 0:
    print("for run id : {:,}\n".format(run_id))
    print('\n\nQC Pass: Urbansim units matches board report capacity assumptions\n\n')
else:
    print("\nfor run id : {:,}\n".format(run_id))
    print('\n\nQC Fail: Urbansim units are not less than the board report capacity assumptions\n\n')
    if num_remaining2016to2035 > 0:
        print('\n\nFailed Cities 2016 to 2035:\n\n')
        print(t2016to2035.loc[t2016to2035.remaining2016to2035<0])
    if num_remaining2036to2050 > 0:
        print('\n\nFailed Cities 2036 to 2050:\n\n')
        print(t2016to2050.loc[t2016to2050.remaining2036to2050<0])
    if num_remaining > 0:
        print('\n\n\n\nFailed Cities Total over entire forecast: 2016 to 2050:\n\n')
        print(tTotal.loc[tTotal.remaining<0])
    print('\n\n\nQC Fail\n\n')


for run id : 411



QC Fail: Urbansim units are not less than the board report capacity assumptions




Failed Cities 2016 to 2035:


         jur_name  twg2016to2035  urb2016to2035  remaining2016to2035
jur_id                                                              
12      Oceanside           6900           7119                 -219
16         Santee           3300           3306                   -6


Failed Cities 2036 to 2050:


              jur_name  twg2016to2035  urb2016to2035  twg2036to2050  \
jur_id                                                                
1             Carlsbad           6600           5173           2100   
2          Chula Vista          27700           9214          14200   
3             Coronado            200            106            200   
6            Encinitas           2500           1442           1100   
8       Imperial Beach           3600           2894            300   
9              La Mesa           9300           7653        